In [ ]:
!pip install tflite

In [1]:
#import tensorflow as tf
import tflite_runtime.interpreter as tflite
#from tensorflow.keras.utils import load_img, img_to_array
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:f}'.format})

In [2]:
def get_label_encoder():
    encoder = LabelEncoder()
    encoder.classes_ = np.load('./best_classes.npy')
    return encoder

In [ ]:
print(tf.__version__)

In [3]:
# Load TFLite model and allocate tensors.
# import cv2

TFLITE_MODEL="./vecchio_modello_nuovo_dataset_55fotoclasse_hue.tflite"
interpreter = tflite.Interpreter(TFLITE_MODEL)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load image
#img = load_img("./images/pikachu_peluche.jpg", target_size=(224, 224))
#plt.imshow(img)
img = Image.open("./images/pikachu_peluche.jpg")
img = img.resize((224, 224), Image.ANTIALIAS)
img = np.asarray(img, dtype=np.float32)
#img = img_to_array(img, dtype=np.float32)
img /= 255
img = np.expand_dims(img, axis=0)
#img = tf.transpose(img, [0, 3, 1, 2])
print(img.shape)

input_tensor = np.array(img, dtype=np.float32)
# Load the TFLite model and allocate tensors.
interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()

# Get output
output_data = interpreter.get_tensor(output_details[0]['index'])

(1, 224, 224, 3)


In [4]:
label_encoder = get_label_encoder()

In [5]:
# tflite_runtime.interpreter as tflite method
results = np.squeeze(output_data, axis=0)
top_k_idx = np.argsort(results)[-5:][::-1]
top_k_values = results[top_k_idx]
top_k_labels = label_encoder.inverse_transform(top_k_idx)
print (top_k_labels, top_k_values)
# print( "\n".join([f"{i}: {v:.3f}" for i, v in zip(top_k_labels, top_k_values)]))

['Pikachu' 'Alakazam' 'Sandshrew' 'Staryu' 'Metapod'] [0.999246 0.000401 0.000206 0.000057 0.000028]


In [ ]:
import tensorflow as tf

In [ ]:
# Tensorflow method
label_encoder = pc.get_label_encoder()
(top_k_scores, top_k_idx) = tf.math.top_k(output_data, 1)
top_k_scores = np.squeeze(top_k_scores.numpy(), axis=0)
top_k_idx = np.squeeze(top_k_idx.numpy(), axis=0)
top_k_labels = label_encoder.inverse_transform(top_k_idx)
print (top_k_labels, top_k_scores)